In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import VotingRegressor
import seaborn as sns


# Data Cleaning and Preprocessing

In [2]:
train= pd.read_csv("C:/Users/Wommack/Downloads/Housing_dataset_train.csv")
#train.head()

In [3]:
test= pd.read_csv("C:/Users/Wommack/Downloads/Housing_dataset_test.csv")
#test.head()

In [4]:
sample_submission= pd.read_csv("C:/Users/Wommack/Downloads/Sample_submission (2).csv")
#sample_submission.head()

In [5]:
#train.info()

In [9]:
train.head()

,ID,loc,title,bedroom,bathroom,parking_space,price
0,3583,Katsina,Semi-detached duplex,2.0,2.0,1.0,1149999.565
1,2748,Ondo,Apartment,NaN,2.0,4.0,1672416.689
2,9261,Ekiti,NaN,7.0,5.0,NaN,3364799.814
3,2224,Anambra,Detached duplex,5.0,2.0,4.0,2410306.756
4,10300,Kogi,Terrace duplex,NaN,5.0,6.0,2600700.898


In [8]:
train.describe()

,ID,bedroom,bathroom,parking_space,price
count,14000.000000,12201.000000,12195.000000,12189.000000,1.400000e+04
mean,4862.700357,4.308171,3.134235,3.169825,2.138082e+06
std,3818.348214,2.441165,2.035950,1.599415,1.083057e+06
min,0.000000,1.000000,1.000000,1.000000,4.319673e+05
25%,1672.750000,2.000000,1.000000,2.000000,1.393990e+06
50%,3527.000000,4.000000,2.000000,3.000000,1.895223e+06
75%,8011.250000,6.000000,5.000000,4.000000,2.586699e+06
max,12999.000000,9.000000,7.000000,6.000000,1.656849e+07


In [ ]:
# Data Cleaning and Preprocessing

train= pd.read_csv("C:/Users/Wommack/Downloads/Housing_dataset_train.csv")
#train.head()

In [ ]:
test= pd.read_csv("C:/Users/Wommack/Downloads/Housing_dataset_test.csv")
#test.head()


In [ ]:
sample_submission= pd.read_csv("C:/Users/Wommack/Downloads/Sample_submission (2).csv")
#sample_submission.head()


In [ ]:
#train.info()

In [ ]:
#train.head()

In [ ]:
#train['title'].value_counts()

In [ ]:
#train['bedroom'].value_counts()

In [ ]:
#train['bathroom'].value_counts()

In [ ]:
#train['parking_space'].value_counts()

In [ ]:
#train[train['loc'].isnull()]

In [ ]:
train2= train.copy()
train2=train2.dropna()

test2= test.copy()
test2=test2.dropna()

In [ ]:
#train2.info()

In [ ]:
#label encoding the data
le= LabelEncoder()

train2['loc']= le.fit_transform(train2['loc'])
test2['loc']= le.transform(test2['loc'])



train2['title']= le.fit_transform(train2['title'])
test2['title']= le.transform(test2['title'])
#train2

In [ ]:
train2['bedroom']= train2['bedroom'].fillna(train2['bedroom'].median())
train2['bathroom']= train2['bathroom'].fillna(train2['bathroom'].median())
train2['parking_space']= train2['parking_space'].fillna(train2['parking_space'].median())

# Data Visualization¶

In [ ]:
#target data['price'] visualization
plt.hist(train2['price'], bins=30)
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.title('Histogram of House Prices')
plt.show()

Checking How Correlated The Features Are

In [ ]:
#visualizing the correlation of features
correlation_matrix = train2.corr()
sns.heatmap(correlation_matrix, annot=True)
plt.show()

In [ ]:
feature_cols= ['loc','title','bedroom', 'bathroom', 'parking_space']

Transforming The Data To Handle Skewness

In [ ]:
# Create a StandardScaler object
scaler = StandardScaler()

# Use the scaler to standardize the feature columns
train2[feature_cols] = scaler.fit_transform(train2[feature_cols])

# IMPORTANT: Use the same scaler to standardize the test data (do not fit again)
test2[feature_cols] = scaler.transform(test2[feature_cols])

# Model Building

In [ ]:
# Split the data into training and validation sets
from sklearn.model_selection import train_test_split
X_train_full, X_val, y_train_full, y_val = train_test_split(train2[feature_cols], train2['price'], test_size=0.2, random_state=23)

In [ ]:
#defining the model
model= RandomForestRegressor()


model.fit(X_train_full, y_train_full)

# Model Evaluation and Validation

In [ ]:
# Define models
models = ['Linear Regression', LinearRegression(),
    ('Random Forest', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('XGBoost', XGBRegressor(random_state=42)),
    ('Decision Tree', DecisionTreeRegressor(random_state=42)),
    ('Gradient Boosting', GradientBoostingRegressor(random_state=42)),
    ('SVM', SVR())
]


# Train and evaluate models
for model_info in models:
    # Check that model_info is a tuple of length 2
    if not isinstance(model_info, tuple) or len(model_info) != 2:
        print(f'Skipping invalid model_info: {model_info}')
        continue

    name, model = model_info
    
    print(f'----- {name} -----')
   
    # Train the model
    model.fit(X_train_full, y_train_full)

    # Make predictions
    y_pred_train = model.predict(X_train_full)
    y_pred_val = model.predict(X_val)
    
    # Compute metrics
    rmse_train = np.sqrt(mean_squared_error(y_train_full, y_pred_train))
    rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
    r2_val = r2_score(y_val, y_pred_val)

    print(f'Training RMSE: {rmse_train}')
    print(f'Validation RMSE: {rmse_val}')
    print(f'Validation R^2: {r2_val}\n')

# Feature Selection

In [ ]:
# Fit a RandomForestRegressor to the data
Rf = RandomForestRegressor()
Rf.fit(X_train_full, y_train_full)

# Get the feature importances
importances = Rf.feature_importances_
importances

# Convert feature importances into a pandas dataframe for better visualization
feature_importances = pd.DataFrame({'feature': X_train_full.columns, 'importance': importances})

# Print feature importances
print(feature_importances.sort_values(by='importance', ascending=False))

In [ ]:
# Visualize feature importances
plt.figure(figsize=(10,6))
sns.barplot(x='importance', y='feature', data=feature_importances)
plt.title('Feature importances given by RandomForestRegressor')
plt.show()

Using The Most Important Features

In [ ]:
feature_cols2= ['bedroom', 'title', 'loc']

In [ ]:
X_train, X_val2, y_train, y_val2 = train_test_split(train2[feature_cols2], train2['price'], test_size=0.2, random_state=23)

Model Evaluation and Validation Using Features From Feature Selection Process

In [ ]:
# Train and evaluate models
for model_info in models:
    # Check that model_info is a tuple of length 2
    if not isinstance(model_info, tuple) or len(model_info) != 2:
        print(f'Skipping invalid model_info: {model_info}')
        continue

    name, model = model_info
    
    print(f'----- {name} -----')
   
    # Train the model
    model.fit(X_train, y_train)

    # Make predictions
    y_pred_train2 = model.predict(X_train)
    y_pred_val2 = model.predict(X_val2)
    
    # Compute metrics
    rmse_train2 = np.sqrt(mean_squared_error(y_train, y_pred_train2))
    rmse_val2 = np.sqrt(mean_squared_error(y_val2, y_pred_val2))
    r2_val_2 = r2_score(y_val2, y_pred_val2)

    print(f'Training RMSE: {rmse_train2}')
    print(f'Validation RMSE: {rmse_val2}')
    print(f'Validation R^2: {r2_val_2}\n')

# Using Ensembling Technique

In [ ]:
# Ensemble

# Define models
models2 = [
    ('Random Forest', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('XGBoost', XGBRegressor(random_state=42)),
    #('Gradient Boosting', GradientBoostingRegressor(random_state=42))
]


ensemble = VotingRegressor(estimators=models2)
ensemble.fit(X_train, y_train)
y_pred_val_ensemble = ensemble.predict(X_val2)

# Compute metrics for the ensemble
rmse_val_ensemble = np.sqrt(mean_squared_error(y_val2, y_pred_val_ensemble))
r2_val_ensemble = r2_score(y_val, y_pred_val_ensemble)

print(f'----- VotingRegressor Ensemble -----')
print(f'Validation RMSE: {rmse_val_ensemble}')
print(f'Validation R^2: {r2_val_ensemble}')

In [ ]:
# Define a second-stage regressor
second_stage_regressor = XGBRegressor(random_state= 42)

# Define the ensemble
ensemble = StackingRegressor(estimators=models2, final_estimator=second_stage_regressor)
ensemble.fit(X_train, y_train)
y_pred_val_ensemble = ensemble.predict(X_val2)

# Compute metrics for the ensemble
rmse_val_ensemble = np.sqrt(mean_squared_error(y_val2, y_pred_val_ensemble))
r2_val_ensemble = r2_score(y_val2, y_pred_val_ensemble)

print(f'----- StackingRegressor Ensemble -----')
print(f'Validation RMSE: {rmse_val_ensemble}')
print(f'Validation R^2: {r2_val_ensemble}')


In [ ]:
# Perform cross validation
cv_scores = cross_val_score(ensemble, X_train, y_train, cv=5)

# Print cross-validation scores
print(f'Cross-validation scores: {cv_scores}')
print(f'Mean cross-validation score: {cv_scores.mean()}')

Submission

In [ ]:
test3=test2.copy()
test3= test3.drop('ID', axis=1)
test3.info()

In [ ]:
#creating the final model
final_model = XGBRegressor()

final_model.fit(X_train_full, y_train_full)

# Make predictions on the test data
test_pred = final_model.predict(test3)

# Create a DataFrame for your submission
submission = pd.DataFrame({
    'ID': test['ID'], 
    'price': test_pred
})

# Save the submission DataFrame as a CSV file
submission.to_csv('submission.csv', index=False)

In [ ]:
submission.head()

# Conclusion

The baseline(when all the null values are dropped from the train dataset) appears to give a good result. XGBoostRegressor appears to be the best model algorithm, giving root mean square error of 88.2% and Using all the input features, except the 'ID' feature shows to be a more reliable set of data for this task, better than the features produced by additional techniques.

In [ ]:
import pickle
#file = open('final_model.pkl','wb')
#pickle.dump(final_model,file)